# **<center style="color: green;">Desafío - Transformación y manipulación de datos (Parte II)</center>**

### **Nombre:** Cristian Tobar Morales ###
### **Curso:** Programación con Python para el análisis de datos (G101) ###

### **Descripción**
La **empresa ADL** se dedica a la venta de productos a nivel regional en los Estados Unidos,
con equipos de ventas que operan en diferentes estados. La compañía ha establecido un
sistema de comisiones para incentivar a sus equipos de ventas a preferir la venta de
productos de alto margen antes que los de bajo margen de utilidad.
Como Analista de Datos, le solicitan analizar información sobre las ventas realizadas por
los equipos de ventas de ADL, incluyendo detalles sobre los productos vendidos, los
clientes, las fechas de los pedidos y los plazos de entrega.
Para resolver este desafío deberás cargar datos desde Hojas de Excel del archivo
US_Regional_Sales_Data.xlsx. En base a esto, debe generar una visualización de datos
como la que se muestra en las hojas Reporte1 y Reporte2 del archivo reporte.xlsx.
Para cumplir con lo solicitado, puedes aplicar los siguientes pasos:

1. Carga los datos de cada pestaña del archivo US_Regional_Sales_Data.xlsx en un
DataFrame independiente. (hint: Puedes obtener los nombres de las hojas usando
pd.ExcelFile(archivo).sheet_names)

In [ ]:
# Importar libreria
import pandas as pd

# Ruta del archivo
ar = "../Desafío_Transformación_Manipulación_Datos/db/US_Regional_Sales_Data.xlsx"

# Nombre de de las hojas
nombre_hojas = pd.ExcelFile(ar).sheet_names

#Se crea una diccionario para almacenar los Dataframe
dataframes = {}

#Cargar cada hoja al diccionario
for hoja in nombre_hojas:
    dataframes[hoja] = pd.read_excel(ar, sheet_name=hoja)

# Mostrar los nombres de los DataFrames
for nombre_hoja, df in dataframes.items():
    print(f'DataFrame para la hoja: {nombre_hoja}')

DataFrame para la hoja: Sales Orders Sheet
DataFrame para la hoja: Customers Sheet
DataFrame para la hoja: Store Locations Sheet
DataFrame para la hoja: Products Sheet
DataFrame para la hoja: Sales Team Sheet


2. Cruza todos los DataFrames usando validación many_to_one, y guarda el resultado
en una nueva variable llamada df_base.

In [12]:
# Los dataframes
df_sales_orders = dataframes['Sales Orders Sheet']
df_customers = dataframes['Customers Sheet']
df_store_locations = dataframes['Store Locations Sheet']
df_products = dataframes['Products Sheet']
df_sales_team = dataframes['Sales Team Sheet']

# Cruzar los dataframes
# 1: Se cruza df_sales_orders con df_customers usando 'CustomerID'
df_base = df_sales_orders.merge(df_customers, on = '_CustomerID', validate = 'many_to_one')

# Luego cruzamos el resultado con df_store_locations usando 'StoreID'
df_base = df_base.merge(df_store_locations, on = '_StoreID', validate = 'many_to_one')

# Luego cruzamos el resultado con df_products usando 'ProductID'
df_base = df_base.merge(df_products, on = '_ProductID', validate = 'many_to_one')

# Finalmente, cruzamos el resultado con df_sales_team usando 'SalesPersonID'
df_base = df_base.merge(df_sales_team, on = '_SalesTeamID', validate = 'many_to_one')

# Mostrar los resultados
df_base.head()


,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,...,AreaCode,Population,Household Income,Median Income,Land Area,Water Area,Time Zone,Product Name,Sales Team,Region
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,...,631,213776,68789,80327,135481314,160302131,America/New York,Dining Furniture,Joshua Bennett,Northeast
1,SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,...,913,186515,74830,72463,194562941,1309517,America/Chicago,Wreaths,Paul Holmes,Midwest
2,SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,...,734,117070,47179,55990,72727009,2254528,America/Detroit,Stemware,Samuel Fowler,Midwest
3,SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,...,203,130322,49771,37192,48407373,3739487,America/New York,Accessories,Carlos Miller,West
4,SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,...,203,108802,40213,40467,73880017,1086045,America/New York,Candles,Joe Price,Northeast


3. Agrega las siguientes columnas a df_ base:

In [18]:
# Verificar si las columnas son del tipo datetime
print("OrderDate: ", df_base['OrderDate'].dtype)
print("ProcuredDate: ", df_base['ProcuredDate'].dtype)
print("ShipDate: ", df_base['ShipDate'].dtype)
print("DeliveryDate: ", df_base['DeliveryDate'].dtype)

OrderDate:  datetime64[ns]
ProcuredDate:  datetime64[ns]
ShipDate:  datetime64[ns]
DeliveryDate:  datetime64[ns]


In [ ]:
# Se agrega la columna ProcurementDays
df_base['ProcurementDays'] = (df_base['OrderDate'] - df_base['ProcuredDate']).dt.days #.days puede ser útil para calcular la diferencia en días entre dos fechas

# Se agrega la columna ShippingDays
df_base['ShippingDays'] = (df_base['ShipDate'] - df_base['OrderDate']).dt.days

# Se agrega la columna DeliveryDays
df_base['DeliveryDays'] = (df_base['DeliveryDate'] - df_base['ShipDate']).dt.days

# Se agrega la columna CustomerDays
df_base['CustomerDays'] = df_base['ShippingDays'] + df_base['DeliveryDays']

4. Agrega una nueva columna discreta al df_base, con el nombre CustomerDaysInterval
que clasifique los valores de la columna CustomerDays en los siguientes intervalos:
○

In [24]:
# 4. Discretización de la columna CustomerDays
# Definimos los intervalos de tiempos de entrega en días
# Discretizar los datos en intervalos
intervalos = [0, 15, 30, 45, 60, 75, 90]

# Etiquetas para las categorías
etiquetas = ["0 to 15 days", "15 to 30 days", "30 to 45 days", "45 to 60 days", "60 to 75 days", "75 to 90 days"]

# Discretizar y devolver valores en lugar de intervalos
df_base["CustomerDaysInterval"] = pd.cut(df_base["CustomerDays"], bins=intervalos, labels=etiquetas)

# Mostramos el DataFrame con la nueva columna
df_base.head()

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,...,Water Area,Time Zone,Product Name,Sales Team,Region,ProcurementDays,ShippingDays,DeliveryDays,CustomerDays,CustomerDaysInterval
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,...,160302131,America/New York,Dining Furniture,Joshua Bennett,Northeast,151,14,5,19,15 to 30 days
1,SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,...,1309517,America/Chicago,Wreaths,Paul Holmes,Midwest,151,22,10,32,30 to 45 days
2,SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,...,2254528,America/Detroit,Stemware,Samuel Fowler,Midwest,151,21,10,31,30 to 45 days
3,SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,...,3739487,America/New York,Accessories,Carlos Miller,West,151,2,5,7,0 to 15 days
4,SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,...,1086045,America/New York,Candles,Joe Price,Northeast,51,16,10,26,15 to 30 days


5. Utiliza una tabla pivote para mostrar el conteo de órdenes agrupando por Sales
Team en las filas y CustomerDaysInterval en las columnas. Este reporte debe
cuadrar con los datos de la pestaña Reporte1 del archivo reportes.xlsx

In [38]:
# Crear la tabla pivote que muestra la cantidad de órdenes por SalesTeam y categoría de entrega
pivot_table = pd.pivot_table(df_base, values='OrderNumber', index='Sales Team', columns='CustomerDaysInterval', aggfunc='count', margins=True, margins_name='All')

# Mostrar la tabla pivote
print("Tabla pivote de conteo de órdenes:")
display(pivot_table)


Tabla pivote de conteo de órdenes:


C:\Users\crist\AppData\Local\Temp\ipykernel_12536\2783739172.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_table = pd.pivot_table(df_base, values='OrderNumber', index='Sales Team', columns='CustomerDaysInterval', aggfunc='count', margins=True, margins_name='All')


CustomerDaysInterval,0 to 15 days,15 to 30 days,30 to 45 days,45 to 60 days,60 to 75 days,75 to 90 days,All
Sales Team,,,,,,,
Adam Hernandez,81,179,42,0.0,0.0,0.0,302
Anthony Berry,76,181,41,0.0,0.0,0.0,298
Anthony Torres,81,162,45,0.0,0.0,0.0,288
Carl Nguyen,96,178,40,0.0,0.0,0.0,314
Carlos Miller,65,148,34,0.0,0.0,0.0,247
Chris Armstrong,92,156,44,0.0,0.0,0.0,292
Donald Reynolds,97,163,36,0.0,0.0,0.0,296
Douglas Tucker,87,145,45,0.0,0.0,0.0,277
Frank Brown,85,150,40,0.0,0.0,0.0,275
